Enabling image
=========================
Here we show agent one example from its own README file and do online research to implement a new multimodal functionality

In [1]:
from lm_agent.agent import Agent
agent = Agent(model='gpt-4', work_dir='gpt-4', config={'command': {'reader': {'summarization_model': 'gpt-4'}}})

### First show the example and ask it to code up the new command


In [2]:
print(agent.instruct("""Read the section 'How to implement a new command' from 
`https://raw.githubusercontent.com/virtualzx-nad/easy_llm_agents/main/README.md` to get the code and instructions for implementing new commands verbatim,
then implement command to get weather forecast for the next 5 days.
If you choose to use an API, make sure you read the doc to understand the parameters and returns structure, and think about what need to be passed and returned for you command.
I cannot provide you with API keys due to privacy restrictions, but the code can directly retrieve the API keys from `self.metadata`. 
Do not use APIs that are not free, and tell me what API keys need to be passed into metadata.  
Note the code, then verify that modules are lazily imported in methods rather than at the beginning of the script before submit the code.
The command will be created or updated as soon as you submit the code, but make sure to notify me and get approval before you start using them."""))

<Permissive Overseer>think requested. "Plan steps to achieve goal". GRANTED.
Creating think with config {'default_model': <lm_agent.models.openai.GPT4 object at 0x7f8a281b2920>}
<Permissive Overseer>delegate requested. "Read the section 'How to implement a new command'". GRANTED.
Creating delegate with config {'default_model': <lm_agent.models.openai.GPT4 object at 0x7f8a281b2920>, 'parent_config': {'command': {'reader': {'summarization_model': 'gpt-4'}}}}
<Command think> {'thoughts': ["Read the section 'How to implement a new command' from the provided URL", 'Implement a command to get weather forecast for the next 5 days', 'Choose a free API and read its documentation', 'Retrieve API keys from self.metadata', 'Determine what API keys need to be passed into metadata', 'Verify that modules are lazily imported in methods', 'Notify user and get approval before using the command'], 'notes': []}
<Command delegate> {'num_tasks': 1}
<Command delegate> {'name': '5eafdb82', 'instruction': "Rea

### Add API keys per its request
Add the requested API key to metadata.  Then check if the new command is now in command list


In [48]:
test = """
import requests

class WeatherForecastCommand:
    def __init__(self, metadata):
        self.api_key = metadata['api_keys']['openweathermap']

    def get_forecast(self, location):
        url = f"https://api.openweathermap.org/data/2.5/forecast?q={location}&units=metric&cnt=5&appid={self.api_key}"
        response = requests.get(url)
        return response.json()

metadata = {'api_keys': {'openweathermap': 'your_api_key_here'}}
weather_forecast = WeatherForecastCommand(metadata)
forecast_data = weather_forecast.get_forecast('London')
print(forecast_data)
"""

In [57]:
import ast

script = test
globals = {}
locals = {}
print('GLOBALS: ', globals)
print('LOCALS: ', locals)
stmts = list(ast.iter_child_nodes(ast.parse(script)))

if isinstance(stmts[-1], ast.Expr):
    print('BRANCH1')
    # the last one is an expression and we will try to return the results
    # so we first execute the previous statements
    if len(stmts) > 1:
        m=(ast.Module(body=stmts[:-1], type_ignores=[]))
        exec(ast.unparse(ast.Module(body=stmts[:-1], type_ignores=[])), globals, locals)
    # then we eval the last one
    print(eval(compile(ast.Expression(body=stmts[-1].value), filename="<ast>", mode="eval"), globals, locals))
else:
    print('BRANCH2')

    # otherwise we just execute the entire code

    print( exec(script, globals, locals))

GLOBALS:  {}
LOCALS:  {}
BRANCH1


NameError: name 'requests' is not defined

In [61]:
g={}
exec(ast.unparse(m), g, g)

In [5]:
agent.config.setdefault('command', {}).get('reader', {}).copy()

{'summarization_model': 'gpt-4'}

### Now ask it to run the new command
We told it to not run it yet because I need to go register for an API key first;  Now it is in metadata we can tell it to continue.
Then ask the question and see it uses the command

In [3]:
print(driver.send("""OK I have added the API key to the metadata.  
Now you can invoke command `weather_forecast`. 
What's the weather in San Francisco the day after tomorrow?"""))

APIConnectionError from OpenAI.  Connection was reset.  Waiting 10s to retry. 
<Command self_note> {'thoughts': ['Invoke the `weather_forecast` command.', 'Find the weather in San Francisco the day after tomorrow.']}
<Command delegate> {'num_tasks': 1}
<Command delegate> {'name': '8b5c3bc6', 'instruction': 'Use the OpenWeatherMap API and the API key in `self.metadata` to get the 5-day weather forecast for San Francisco.', 'context': 'The user wants to know the weather in San Francisco the day after tomorrow. To provide this information, a 5-day weather forecast is needed.\n'}
<Command delegate> {'worker_name': '8b5c3bc6', 'command': 'self_note', 'task': <easy_llm_agents.commands.self_note.SelfNoteCommand object at 0x7f6554a65630>, 'data': {'thoughts': ['The user wants to get the 5-day weather forecast for San Francisco using the OpenWeatherMap API and the API key in `self.metadata`.']}}
<Command delegate> {'worker_name': '8b5c3bc6', 'command': 'python', 'task': <easy_llm_agents.command

### Bugs discovered!

When it runs the code it didn't run.  This is fairly common for GPT generated code.  So it tries again, then again. 
Third time was the charm though!  And it was able to report the weather.  It's gonna get cold!

### Under the hood

Here are the full language chain including the tooling part

In [ ]:
from lm_agent import utils


In [5]:
for entry in conv.history:
    print(f"{entry['role']}: {entry['content']}")

user: 
user instruction: You are in Github repo `easy_llm_agents` of user `virtualzx-nad`.
Read the section 'How to implement a new command' verbatim from `README.md` and implement command to get weather forecast for the next 5 days.
If you choose to use an API, make sure you read the doc to understand the parameters and returns structure, and think about what need to be passed and returned for you command.
I cannot provide you with API keys due to privacy restrictions, but the code can directly retrieve the API keys from `self.metadata`. 
Do not use APIs that are not free, and tell me what API keys need to be passed into metadata.  
Note the code, then verify that modules are lazily imported in methods rather than at the beginning of the script before submit the code.
The command will be created or updated as soon as you submit the code, but make sure to notify me and get approval before you start using them.

assistant: [
  {
    "command": "search",
    "summary": "Find the GitHub r